# 5.6 Affine/Softmax层的实现

## 5.6.2 批版本的Affine层

正向传播时，偏置被加到X*W的各个数据上

In [2]:
import numpy as np

X_dot_W = np.array([[0, 0, 0], [10, 10, 10]])
B = np.array([1, 2, 3])
print(X_dot_W)
print(X_dot_W + B)

[[ 0  0  0]
 [10 10 10]]
[[ 1  2  3]
 [11 12 13]]


**Question**

- 反向传播时，各个数据的反向传播的值需要汇总为偏置的元素

Y = X * W + B

(N * M) = (N * D) (D * M) + (1 * M)

实际是

Y = X * W + I * B

(N * M) = (N * D) (D * M) + (N * 1) (1 * M)

dB = I.T * Y

(1 * M) = (1 * N) (N * M)

- [如何理解反向传播？（知乎）](https://www.zhihu.com/question/27239198?rf=24827633)

In [3]:
# # affine的实现
# # common/layers.py中Affine为四维张量情形

# class Affine:

#     def __init__(self, W, b):
#         self.W = W
#         self.b = b
#         self.x = None
#         self.dW = None
#         self.db = None
    
#     def forward(self, x):
#         self.x = x
#         out = np.dot(x, self.W) + self.b

#         return out
    
#     # 只输出dx???
#     def backward(self, dout):
#         dx = np.dot(dout, self.W.T)
#         self.dW = np.dot(self.x.T, dout)
#         self.db = np.sum(dout, axis=0)

#         return dx

## 5.6.3 Softmax-with-Loss层

In [ ]:
# 批处理版本
import sys, os
sys.path.append(os.pardir)
from common.functions import *

class SoftmaxWithLoss:

    def __init__(self):
        # 损失
        self.loss = None
        # softmax的输出
        self.y = None
        # 监督数据(one-hot vector)
        self.t = None

    def forward(self, x, t):
        self.t = t
        self.y = softmax(x)
        self.loss = cross_entropy_error(self.y, self.t)

        return self.loss
    
    def backward(self, dout=1):
        batch_size = self.t.shape[0]
        dx = (self.y - self.t) / batch_size
        
        return dx